# PD1–TCR Proteomics Analysis (Jurkat)

This notebook auto-detects whether per-sample or aggregated columns are present. Place your data files in the project root or in `data/`.

In [ ]:

from pathlib import Path
import pandas as pd
import numpy as np

PROT_XLSX = Path("tp reordered (1).xlsx") if Path("tp reordered (1).xlsx").exists() else Path("data") / "tp reordered (1).xlsx"
META_CSV  = Path("meta_data (1).csv")      if Path("meta_data (1).csv").exists()      else Path("data") / "meta_data (1).csv"
OUTDIR    = Path("outputs")
OUTDIR.mkdir(parents=True, exist_ok=True)

PROT_XLSX, META_CSV, OUTDIR


In [ ]:

%load_ext autoreload
%autoreload 2
import sys
sys.path.append(".")
from PD1_TCR_proteomics_pipeline import (
    load_metadata, load_proteomics_excel, guess_id_column,
    detect_sample_columns_from_meta, detect_condition_columns,
    log2_transform, median_normalize, filter_missing_by_row, impute_knn,
    to_long_with_meta, pca_plot, per_timepoint_ttest,
    aggregated_long, aggregated_fold_changes, run_pipeline
)

meta = load_metadata(META_CSV)
prot = load_proteomics_excel(PROT_XLSX)
meta.head(), prot.head(), prot.shape, meta.shape


In [ ]:

cols = pd.DataFrame({"column": prot.columns})
cols.to_csv(OUTDIR / "proteomics_columns.csv", index=False)
cols.head(20)


In [ ]:

run_pipeline(PROT_XLSX, META_CSV, OUTDIR, max_missing_frac=0.5)


In [ ]:

import os, pandas as pd
[f for f in os.listdir(OUTDIR)]


In [ ]:

import pandas as pd, os
da_all = OUTDIR / "DA_all_times.csv"
ag_fc  = OUTDIR / "aggregated_log2FC_by_time.csv"
if da_all.exists():
    display(pd.read_csv(da_all).head(10))
if ag_fc.exists():
    display(pd.read_csv(ag_fc).head(10))


**Notes**: If you have per-sample columns, you'll get FDR-adjusted results per timepoint. If you only have aggregated columns, you'll get per-timepoint log2 fold-changes.